## this version should be an upggrade of the previous one where I find a better way to give rewards. Instead of an ordinary TD(0) update as I used in the previous version I should now use a n-step method where I dont give rewards before every bid is given for an hour. 

In [1]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, weather, timeframes, rl_utils, met_api, nve_api
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict, Counter
import random
import sklearn

In [2]:
# GLOBAL VARIABLES:
tf = timeframes.one_week
areas = ["NO5"]


In [3]:
L, M, H = utils.get_all_sets(timeframe= tf, areas = areas)
F, freq_data, _ = utils.get_frequency_sets(tf= tf, M =M, H= H)
L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m = utils.get_parameters(L = L, M = M, H = H)
Ir_hlm, Ia_hlm, Va_hm = utils.get_income_dictionaries(H=H, L = L, M = M, freq_data= freq_data, Fu_h_l= Fu_h_l, Fd_h_l= Fd_h_l, P_h_m= P_h_m, Vp_h_m= Vp_h_m, F = F, markets_dict = {market.name : market for market in M}, timeframe = tf, areas = areas)
compatible_dict = utils.get_compatibility_dict(L = L ,M = M, index = False)

/Users/sandermeland/Documents/UIB/Master/master-kode/code_map/utils.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = "Time", inplace = True)
/Users/sandermeland/Documents/UIB/Master/master-kode/code_map/utils.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = "Time", inplace = True)


In [4]:
sup_market_names = ["FCR", "aFRR"]
markets = [market for market in M if sup_market_names[0] in market.name  or sup_market_names[1] in market.name]
exp_price_dict, exp_vol_dict = rl_utils.get_expected_prices_and_volumes_dict(bid_timeframe= H, markets= markets)
norm_exp_price_dict = rl_utils.normalize_dict_vals(exp_price_dict, norm_method= "min_max")
norm_exp_vol_dict = rl_utils.normalize_dict_vals(exp_vol_dict, norm_method= "min_max")
#weather_data = weather.get_weather_data(tf= tf, areas = ["NO5"])
#norm_w_df = rl_utils.normalize_weather_data(weather_data= weather_data)


In [5]:
fyllingsgrad_df = nve_api.get_fyllinsgrad_df(areas = areas)

In [6]:
tf_fyllyngsgrad_df = fyllingsgrad_df.loc[fyllingsgrad_df["Dato"].isin(utils.get_timestamps(tf= tf))]

In [7]:
weather_dict = met_api.get_normalized_weather_dfs(reference_tf= timeframes.one_month, usage_tf = tf, areas = areas)


In [8]:
weather_dict.keys()

dict_keys([('air_temperature', 'NO5'), ('sum(precipitation_amount P1D)', 'NO5'), ('wind_speed', 'NO5'), ('cloud_area_fraction', 'NO5')])

In [9]:
air_temp_df = weather_dict[("air_temperature", areas[0])]
wind_speed_df = weather_dict[("wind_speed", areas[0])]
precipitation_df = weather_dict[("sum(precipitation_amount P1D)", areas[0])]
cloud_cover_df = weather_dict[("cloud_area_fraction", areas[0])]


In [10]:
spot_path = "../master-data/spot_data/spot_june_23.csv"
norm_da_df = final_markets.preprocess_spot_data(pd.read_csv(spot_path), year = tf.year, start_month = tf.start_month, end_month = tf.end_month, start_day = tf.start_day, end_day = tf.end_day, start_hour = tf.start_hour, end_hour = tf.end_hour, area = areas[0], normalize= True)


In [11]:
markets = [market for market in M if sup_market_names[0] in market.name  or sup_market_names[1] in market.name]
print([m.name for m in markets])

['FCR_D_D_1_NO5', 'FCR_D_D_2_NO5', 'FCR_N_D_1_NO5', 'FCR_N_D_2_NO5', 'aFRR up_NO5', 'aFRR down_NO5']


In [12]:
"""air_temp_df = met_api.get_wanted_weather_values(df2, elementId= "air_temperature", area = "NO5")
precipitation_df = met_api.get_wanted_weather_values(df2, elementId= "sum(precipitation_amount P1D)", area = "NO5")
wind_speed_df = met_api.get_wanted_weather_values(df2, elementId= "wind_speed", area = "NO5")
cloud_area_df = met_api.get_wanted_weather_values(df2, elementId= "cloud_area_fraction", area = "NO5")"""

'air_temp_df = met_api.get_wanted_weather_values(df2, elementId= "air_temperature", area = "NO5")\nprecipitation_df = met_api.get_wanted_weather_values(df2, elementId= "sum(precipitation_amount P1D)", area = "NO5")\nwind_speed_df = met_api.get_wanted_weather_values(df2, elementId= "wind_speed", area = "NO5")\ncloud_area_df = met_api.get_wanted_weather_values(df2, elementId= "cloud_area_fraction", area = "NO5")'

In [13]:
def get_features(bid_hour : pd.Timestamp, available_assets : [new_meters.PowerMeter], market : final_markets.ReserveMarket, 
                 norm_exp_price_dict : dict, norm_exp_vol_dict : dict, precipitation_df : pd.DataFrame, air_temp_df : pd.DataFrame, 
                 cloud_cover_df : pd.DataFrame, wind_speed_df : pd.DataFrame, norm_da_df : pd.DataFrame, L : [new_meters.PowerMeter], 
                 markets : [final_markets.ReserveMarket]):
    """ Function to get the features for the given hour and market. It is important to use features that will help the model learn which actions to take 
        and update the weights correctly for the given state.
        The features that possibly can be used here are the following:
        - day of week
        - hour of day
        - number of possible assets
        - Day Ahead (DA) price
        - Weather forecast including precipitation, temperature, wind speed and cloud cover
        - Market historical prices
        - Market historical volumes
        - frequency data (historical)

        Will add more features as I go along. Fyllingsgrad is ready to be added, but I only have weekly values which means that it will be the same for all hours in the week.
        
    Args:
        available_assets (new_meters.PowerMeter]): _description_
        hour (pd.Timestamp): _description_
        market (final_markets.ReserveMarket): _description_
        norm_exp_price_dict (dict) : dictionary of the normalized expected prices for each direction and area in the bid_timeframe 
        norm_exp_vol_dict (dict) : of the normalized expected volumes for each direction and area in the bid_timeframe 
        norm_w_df (pd.DataFrame) : normalized weather data 
        norm_da_df (pd.DataFrame) : normalized day ahead prices 
        L ([new_meters.PowerMeter]) : list of all possible assets

    Returns:
        np.array: np.array of the normalized features which depends on which market, hour and available assets are given
    """
    day_of_week = bid_hour.weekday()
    hour_of_day = bid_hour.hour
    expected_price = norm_exp_price_dict[(market.direction, market.area, bid_hour)]
    expected_volume = norm_exp_vol_dict[(market.direction, market.area, bid_hour)]
    #precipitation = norm_w_df["precipitation"].loc[norm_w_df["Time (Local)"] == bid_hour]
    #temperature = norm_w_df["air_temp"].loc[norm_w_df["Time (Local)"] == bid_hour]
    precipitation = precipitation_df.loc[bid_hour.replace(hour= 0, minute= 0, second= 0, microsecond= 0)]
    temperature = air_temp_df.loc[bid_hour]
    wind_speed = wind_speed_df.loc[bid_hour]
    cloud_cover = cloud_cover_df.loc[bid_hour.replace(hour= 0, minute= 0, second= 0, microsecond= 0)]
    da_price = norm_da_df["settlement"].loc[norm_da_df["Time(Local)"] == bid_hour]
    market_nr = (markets.index(market)+1)/6
    
    return np.array([day_of_week/7, hour_of_day/24, expected_price, expected_volume, len(available_assets)/len(L), precipitation, temperature, wind_speed, cloud_cover, da_price.values[0], market_nr])

# info opp til 2 dager tilbake i tid kan være interessant, (en type moving average e.l.), skal jeg da ha det for det markedet som er relevant eller for alle? Skal det være for prisene i markedene, volum, begge? DA priser? Vær?
# uforventede utfall - umms - planlagte utfall ligger tilgjengelig - historiske umms ligger på nordpool. Trenger å vite tilgjegelig kapasitet og produksjon for forskjelige teknologier i hvert prisområde og om endringer er planlagt eller uplanlagt. når man får vite det
# bør adde wind speed og skydekke. Dette sier litt om produksjon fra vind og sol og kan derfor påvirke priser. 
# czc mellom budområdene. Regnes ut ved å se på DA prisen for to dager siden og differanse mellom to budområder. 
# vannstand - hvor data hentes fra NVE. Det er interessant, spesielt å se på om vannstand i noen av kraftverkene er tett opp mot 100% av tilgjengelig kapasitet da de må se seg nødt til å produsere mer for å unngå oversvømmelse.


In [14]:
def initialize_weights(n_features :int , n_actions : int, zeros : bool = True):
    if zeros:
        return np.array([np.zeros((n_features)) for _ in range(n_actions)])
    else:
        return np.array([np.random.normal(0, 0.1, n_features) for _ in range(n_actions)])

In [16]:
def train_nstep_model(epsilon : float, alpha : float, gamma : float, num_episodes : int, L : [new_meters.PowerMeter], M : [final_markets.ReserveMarket], H : [pd.Timestamp], norm_exp_price_dict : dict, norm_exp_vol_dict : dict, weather_dict : dict, norm_da_df : pd.DataFrame, n_actions : int, n_features : int):
    """ Training function to learn how to bid in to the markets due to the bidding constraints and the asset constraints as well as optimizing the weights. 
    The function will learn how to bid in to the markets by updating the weights due to the features. 
    The change from v_4 is that this function will compare bids that are set to the same hour to get a better estimate of the reward.
    
    Args:
        epsilon (float): float number between 0 and 1, says how much the agent should explore
        alpha (float): float number between 0 and 1, also known as the learning rate
        gamma (float): float number between 0 and 1, also known as the discount factor
        num_episodes (int): number of episodes to be ran
        L ([new_meters.PowerMeter]): list of PowerMeter objects
        M ([final_markets.Reservemarket]): list of ReserveMarket objects
        H ([pd.Timestamp]): list of timestamps
        norm_exp_price_dict (dict): dictionary of the normalized expected prices for each direction and area in the bid_timeframe
        norm_exp_vol_dict (dict): dictionary of the normalized expected volumes for each direction and area in the bid_timeframe
        norm_w_df (pd.DataFrame) : normalized weather data in pd.dataframe format
        norm_da_df (pd.DataFrame) : normalized day ahead prices in pd.dataframe format
        n_actions (int): number of actions
        n_features (int): number of features

    Returns:
        bids (dict): dictionary that holds control over the final bids for each market and each hour. The keys are tuples of the market name and the hour and the values are the the assets that were bid for the given market and hour. The values are tuples including the list of assets and the aggregated volume.
        revenues (dict): dictionary that holds the revenue for each episode
        available_assets (dict): dictionary that holds the available assets for each hour
        episode_weights (dict): dictionary that holds the weights for each episode
        action_feature_dict: dictionary that holds the features for each action for each episode
        asset_bids (list): list that holds the bids for each episode. The indexes are the episode number and the values are dataframes that holds the bids for each market and each hour for each episode.
    """
    revenues = {}
    bid_timeframe = H[24:] # the hours where bids can be placed in
    place_bid_hours = [hour for hour in H[:-48] if hour.hour == 7 or hour.hour == 17 or hour.hour == 18] # the hours where bids can be placed from
    bids = {}
    asset_bids = []
    sup_market_names = ["FCR", "aFRR"]
    markets = [market for market in M if sup_market_names[0] in market.name  or sup_market_names[1] in market.name]
    #market_names = [market.name for market in markets]
    # will only use FCR-N, FCR-D and aFRR. The FCR markets are both D-1 and D-2 and the aFRR market is D-1 but it is both up and down
    air_temp_df = weather_dict[("air_temperature", areas[0])]
    wind_speed_df = weather_dict[("wind_speed", areas[0])]
    precipitation_df = weather_dict[("sum(precipitation_amount P1D)", areas[0])]
    cloud_cover_df = weather_dict[("cloud_area_fraction", areas[0])]

    available_assets = {hour: L.copy() for hour in bid_timeframe} 
    #bids = {(market.name, hour): [] for hour in bid_timeframe for market in markets}
    "For each hour, an action should be to either bid in every feasible asset, to bid the minimum volume or to bid nothing"
    (possible_hours, market_name) = rl_utils.get_possible_dates(place_bid_hours[0]) # the market_name is not the full name of the market, but rather a substring of the full name. Since FCR-D and FCR-N has the same deadlines there will be two markets with the same substring
    
    possible_markets = [m for m in markets if market_name in m.name] # will either be one or two markets
    #indexes = [market_names.index(m.name) for m in possible_markets] # the indexes for each market. This is used to slice the Q-table so it is not necessary in this case
    features = get_features(bid_hour = possible_hours[0], available_assets= available_assets[possible_hours[0]], 
                            market = possible_markets[0],norm_da_df=norm_da_df, norm_exp_price_dict= norm_exp_price_dict, 
                            norm_exp_vol_dict= norm_exp_vol_dict, precipitation_df= precipitation_df, cloud_cover_df= cloud_cover_df, 
                            wind_speed_df= wind_speed_df, air_temp_df= air_temp_df, L = L, markets = markets)

    
    action_0 = random.randint(0, n_actions-1)
    action_feature_dict = {}
    weights = initialize_weights(n_actions = n_actions, n_features = n_features, zeros = False)

    # extract hours from H where hour == 7, 17, 18
    epsilon_decay = epsilon/num_episodes
    alpha_decay = alpha/num_episodes
    episode_weights = {}
    for episode_n in range(num_episodes):
        if episode_n > episode_n/10:
            epsilon -= epsilon_decay
            alpha -= alpha_decay
        revenue = 0
        episode_bid_df = pd.DataFrame(columns= ["Market", "Hour", "Asset Count", "Total Flex Volume"])
        episode_bid_dict = {}
        reward_bid_hour_dict = {}
       
        #print(f" weights at the start of episode nr {episode_n}: {weights}")

        available_assets = {hour: L.copy() for hour in bid_timeframe}
        for place_hour in place_bid_hours: # the hours when the bids are placed from
            (possible_hours, market_name) = rl_utils.get_possible_dates(place_hour) # returns the possible hours for the market to place bids in
            if len(possible_hours) != 24:
                print(f"No bids for {place_hour}")
                print(f"possible_hours: {len(possible_hours)}")
                break
            #print(f"possible_hours when bidding: {possible_hours}")
            possible_markets = [m for m in markets if market_name in m.name] # because i am now using only no5 markets, this list should be of length 1 or 2.
            
            for current_market in possible_markets: # the markets that are possible to bid in
                #print(f"current_market : {current_market.name}")
                for bid_hour in possible_hours:
                    chosen_portfolio, reward, flex_vol = rl_utils.make_bid(current_market, bid_hour, action_0, available_assets[bid_hour], compatible_dict) # get portfolio and reward for a bid - will have to figure out a better way to define reward
                    if bid_hour not in reward_bid_hour_dict.keys():
                        reward_bid_hour_dict[bid_hour] = [reward]
                    else:
                        reward_bid_hour_dict[bid_hour].append(reward)

                    available_assets[bid_hour] = [asset for asset in available_assets[bid_hour] if asset not in chosen_portfolio] # remove the assets that were bid from the available assets

                    if len(chosen_portfolio) > 0:
                        episode_bid_df.loc[len(episode_bid_df)] = [current_market.name, bid_hour, len(chosen_portfolio), flex_vol] # add the bid to the episode_bid_df

                    episode_bid_dict[(current_market.name, bid_hour)] = (chosen_portfolio, flex_vol) # add the bid to the bids dictionary
                    
                    new_features = get_features(bid_hour = bid_hour, available_assets= available_assets[bid_hour], 
                                                market = current_market, norm_da_df=norm_da_df, norm_exp_price_dict= norm_exp_price_dict, 
                                                norm_exp_vol_dict= norm_exp_vol_dict, precipitation_df= precipitation_df, cloud_cover_df= cloud_cover_df, 
                                                wind_speed_df= wind_speed_df, air_temp_df= air_temp_df, L = L, markets = markets) # update the features
                    #print(f"weights : {weights}")
                    possible_actions = [np.dot(weights[action], new_features) for action in range(n_actions)] 
                    #print(f"possible_actions: {possible_actions}")
                    new_action = rl_utils.greedy_action(possible_actions, epsilon)
                    action_feature_dict[(episode_n, current_market.name, bid_hour)] = (action_0, new_action, features, new_features)
                    features, action_0 = new_features, new_action
                    revenue += reward
            #print([m.name for m in possible_markets])
            # Instead of updating the weights after each hour, the weights are updated after each market. This is to get a better estimate of the reward since the rewards are given after each of the markets are considererd for an hour.
            if "FCR_N_D_1_NO5" in [m.name for m in possible_markets]: # The FCR D-1 markets has the latest deadline, so after these markets has been considered, all markets has been considered for the given hour.
                #print(f"Possible hours when updating the weights : {possible_hours}")
                for bid_hour in possible_hours: 
                    total_reward = sum(reward_bid_hour_dict[bid_hour])
                    #print(f"total_reward: {total_reward}")
                    for market in markets:
                        if (episode_n, market.name, bid_hour) in action_feature_dict.keys(): # should maybe rearrange the structure of the code so that this if statement is not necessary
                            action_0, new_action, features, new_features = action_feature_dict[(episode_n, market.name, bid_hour)]
                            
                            q_values_next_state = np.array([np.dot(weights[a], new_features) for a in range(n_actions)])
                            
                            
                            weights[action_0] = weights[action_0] + alpha * \
                            (total_reward/255 + gamma * np.max(q_values_next_state) - np.dot(weights[action_0], features)) * features
                            #print(f"weights after updating: {weights}")

        episode_weights[episode_n] = weights.copy()        
        bids[episode_n] = episode_bid_df.sort_values(by = ["Hour"])
        revenues[episode_n] = revenue
        asset_bids.append(episode_bid_dict)
    return bids, revenues, available_assets, episode_weights, action_feature_dict, asset_bids

In [93]:
bids, revenues, available_assets, weights, action_list, asset_bids = train_nstep_model(epsilon = 0.3, alpha = 0.3, gamma = 0.9, num_episodes = 200, L = L, M = M, H = H, norm_exp_price_dict = norm_exp_price_dict, norm_exp_vol_dict = norm_exp_vol_dict, weather_dict= weather_dict, norm_da_df = norm_da_df, n_features= 11, n_actions= 3)

In [ ]:
"""action_dict = {}
for e in range(2):
    action_0s = []
    actions = []
    feature_list = []
    new_feature_list = []    
    for market in markets:
        for hour in H:
            if (e, market.name, hour) in action_list.keys():
                action_0, new_action, features, new_features = action_list[(e, market.name, hour)]
                action_0s.append(action_0)
                actions.append(new_action)
                feature_list.append(features)
                new_feature_list.append(new_features)

    action_dict[e] = (action_0s, actions, feature_list, new_feature_list)
    
    used this to check if the actions were the same for the same hour and market. They were not.
    """

In [90]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

def make_plots_for_sol( sol_name : str = "Optimization", sol_df : pd.DataFrame = None, y_axis_name : str = "Asset Count"):
    """ function to make plot of the results recieved from the wanted model. This function can either plot how many assets that were bid in each market for each hour or the total flex volume that was bid in each market for each hour.

    Args:
        dict_of_dfs (dict): for the optimization model, the results are stored in a dict. The keys are the hours and the values are dataframes that holds the results for each hour.
        sol_name (str, optional): Either "Optimization" or "RL. Defaults to "Optimization".
        rl_df (pd.DataFrame, optional): The results for the RL df are stored in a dataframe and this should be given here if the results from the RL model is wanted. The RL model returns a list of dataframes, and that list with the wanted index (episode) should be given. Defaults to None.
        y_axis (str, optional): Choose which y-axis value that is wanted. Can be either "Asset Count" or "Total Flex Volume". Defaults to "Asset Count".
    """
    
    combined_df = sol_df.copy()
    if sol_name == "Optimization":
        combined_df = combined_df.loc[combined_df["Hour"] >=  H[24]] # only want to plot the hours where bids can be placed in
    # Create a figure with secondary y-axis
    fig = go.Figure()
    # Iterate over each market and add a scatter trace for Asset Count and Total Flex Volume
    for market in combined_df['Market'].unique():
        market_data = combined_df[combined_df['Market'] == market]
        fig.add_trace(go.Scatter(x=market_data['Hour'], y=market_data[y_axis_name],mode='markers', name=f"{y_axis_name} - {market}"))
        
    # Add figure title and layout
    fig.update_layout(title_text=f"{y_axis_name} by Market and Hour from the {sol_name} model")

    # Set x-axis title
    fig.update_xaxes(title_text="Hours")

    # Set y-axes titles
    fig.update_yaxes(title_text= y_axis_name)
    

    # Show the figure
    fig.show()

In [99]:
make_plots_for_sol(sol_name= "RL", sol_df= bids[199])

In [96]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= [i for i in range(len(revenues.keys()))], y= [i for i in revenues.values()], mode= "markers"))
# add title and axis labels
fig.update_layout(title_text="Revenue per episode for RL model", xaxis_title="Episode", yaxis_title="Revenue")
# add a regression line for the revenue values for each episode.
fig.add_trace(go.Scatter(x= [i for i in range(len(revenues.keys()))], y= [i for i in revenues.values()], mode= "lines"))
fig.show()

In [97]:
fig = go.Figure()
for w in range(3):
    fig.add_trace(go.Scatter(x= [i for i in range(len(weights.keys()))], y= [np.sum(i[w]) for i in list(weights.values())], mode= "markers", name= f"weight {w}"))
# add title and axis labels
fig.update_layout(title_text="Weights per episode for RL model", xaxis_title="Episode", yaxis_title="sum of the weights")
fig.show()

In [98]:
for weight in range(3):
    fig = go.Figure()
    for f in range(11):
        fig.add_trace(go.Scatter(x= [i for i in range(len(weights.keys()))], y= [np.sum(i[w, f]) for i in list(weights.values())], mode= "markers", name= f"feature {f} for weight {weight}"))
    fig.update_layout(title_text=f"Weights per episode for RL model for weight {weight}", xaxis_title="Episode", yaxis_title="sum of the weights")
    fig.show()

In [24]:
compatible_list = utils.get_compatibility_dict(L = L, M = M, index = True)

In [64]:
mod, x, y, w, d = utils.run_optimization_model(L = L, M = M, F = F, H = H, Fu_h_l = Fu_h_l, Fd_h_l = Fd_h_l, R_h_l = R_h_l, Vp_h_m = Vp_h_m, Vm_m = Vm_m, R_m = R_m, Ir_hlm = Ir_hlm, Ia_hlm = Ia_hlm, Va_hm = Va_hm, compatible_list= compatible_list, log_filename= "week_mod_no5.log", model_name= "week_mod_no5")

Set parameter LogFile to value "week_mod_no5.log"
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threadsThread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2747808 rows, 1292928 columns and 8383574 nonzerosOptimize a model with 2747808 rows, 1292928 columns and 8383574 nonzeros
Model fingerprint: 0x2ee98c6bModel fingerprint: 0x2ee98c6b
Model has 120014 quadratic objective termsModel has 120014 quadratic objective terms
Model has 1680 quadratic constraintsModel has 1680 quadratic constraints
Variable types: 0 continuous, 1292928 integer (1292928 binary)Variable types: 0 continuous, 1292928 integer (1292928 binary)
Coefficient statistics:Coefficient statistics:
Matrix range     [1e-06, 1e+03]  Matrix range     [1e-06,

## The functions below are to check the revenue for each hour for the optimization model

In [65]:
def get_market_count_df(x : dict, y: dict, w : dict,  H : [pd.Timestamp], L : [new_meters.PowerMeter], M : [final_markets.ReserveMarket]):
    """ function to get a dictionary of the results of the optimization problem.
        The solution is represented as a dataframe for each hour which tells how many assets and how much flex volume is connected to each market for each hour.

    Args:
        x (dict): dictionary of the binary variable which tells if an asset is connected to a market
        L (list(PowerMeter)): list of powermeter objects with the data for each meter within the timeframe
        M (list(ReserveMarket)): list of reservemarket objects with the data for each market within the timeframe
        H (list(pd.TimeStamp)): list of hourly timestamps within the timeframe
        dominant_directions (list(str)): list of the dominant direction for each hour

    Returns:
        dict: the solution of the optimization problem
    """
    
    df = pd.DataFrame(columns=["Market", "Hour", "Asset Count","Total Flex Volume [MWh]", "Total capacity revenue [EUR])", "Total activation revenue [EUR]"])
    for h, hour in enumerate(H):
        for m, market in enumerate(M):
            if y[h, m].X > 0:
                amount_of_assets = sum(x[h, l, m].X for l in range(len(L)))
                capacity_income = sum(x[h, l, m].X * Ir_hlm[h, l, m] for l in range(len(L)))
                activation_income = sum(x[h, l, m].X * Ia_hlm[h, l, m] * w[h,m].X for l in range(len(L)))
                if market.direction == "up":
                    total_flex_volume = sum(x[h, l, m].X * Fu_h_l[h,l] for l in range(len(L)))
                elif market.direction == "down":
                    total_flex_volume = sum(x[h, l, m].X * Fd_h_l[h,l] for l in range(len(L)))
                else:
                    total_flex_volume = 0
                    for l, load in enumerate(L):
                        if load.direction == "up":
                            total_flex_volume += x[h, l, m].X * Fu_h_l[h,l]
                        elif load.direction == "down":
                            total_flex_volume += x[h, l, m].X * Fd_h_l[h,l]
                        else:
                            total_flex_volume += min(x[h, l, m].X * Fu_h_l[h,l], x[h, l, m].X * Fd_h_l[h,l])
                           
                df.loc[len(df)] = [market.name, hour, amount_of_assets, total_flex_volume, capacity_income, activation_income]
    return df

In [66]:
mcd = get_market_count_df(x = x, y =y, w = w, H = H, L = L, M = M)

In [67]:
display(mcd)

,Market,Hour,Asset Count,Total Flex Volume [MWh],Total capacity revenue [EUR]),Total activation revenue [EUR]
0,aFRR up_NO5,2023-06-19 00:00:00+02:00,580.0,3.351549,67.030980,0.0
1,aFRR up_NO5,2023-06-19 05:00:00+02:00,531.0,2.706115,62.240645,0.0
2,aFRR up_NO5,2023-06-19 06:00:00+02:00,529.0,2.383027,52.426594,0.0
3,aFRR up_NO5,2023-06-19 07:00:00+02:00,543.0,2.800745,61.616390,0.0
4,aFRR up_NO5,2023-06-19 08:00:00+02:00,575.0,2.930003,64.460066,0.0
...,...,...,...,...,...,...
223,aFRR down_NO5,2023-06-25 20:00:00+02:00,257.0,2.908685,72.717125,0.0
224,FCR_N_D_1_NO5,2023-06-25 21:00:00+02:00,271.0,1.277727,44.209354,0.0
225,aFRR down_NO5,2023-06-25 21:00:00+02:00,278.0,3.233297,129.331880,0.0
226,aFRR down_NO5,2023-06-25 22:00:00+02:00,528.0,4.190777,167.631080,0.0


In [68]:
print(mcd["Market"].unique())

['aFRR up_NO5' 'aFRR down_NO5' 'FCR_N_D_1_NO5' 'FCR_N_D_2_NO5']


In [69]:
mcd.columns

Index(['Market', 'Hour', 'Asset Count', 'Total Flex Volume [MWh]',
       'Total capacity revenue [EUR])', 'Total activation revenue [EUR]'],
      dtype='object')

In [70]:
sum(mcd["Total activation revenue [EUR]"])

61.134113126764774

In [71]:
sum(mcd["Total capacity revenue [EUR])"])

11828.193289429999

In [72]:
sol_df_sum = sum(mcd["Total capacity revenue [EUR])"])+ sum(mcd["Total activation revenue [EUR]"])

In [73]:
sol_df_sum

11889.327402556764

In [74]:
opt_mod_tot_rev = sum(x[h,l,m].X * (Ir_hlm[h,l,m] + Ia_hlm[h,l,m] * w[h,m].X) for h in range(len(H)) for l in range(len(L)) for m in range(len(M)))

In [75]:
opt_mod_tot_rev

11889.32740255678

In [77]:
max(mcd["Total capacity revenue [EUR])"])

253.2188907600004

In [91]:
make_plots_for_sol(sol_df = mcd, sol_name= "Optimization")

In [92]:
make_plots_for_sol(sol_df = mcd, sol_name= "Optimization", y_axis_name= "Total Flex Volume [MWh]")

In [89]:
for hour in mcd["Hour"].unique():
    # find out if there are any hours which contains three markets
    if len(mcd.loc[mcd["Hour"] == hour]) >= 3:
        print(hour)

2023-06-20 22:00:00+02:00
2023-06-23 12:00:00+02:00
2023-06-23 13:00:00+02:00
2023-06-23 14:00:00+02:00
2023-06-23 15:00:00+02:00
2023-06-23 16:00:00+02:00
